# 금융보험 콜센터 데이터 - 카테고리별 의도 분석

4개 카테고리별 고객의도와 상담사의도 분포를 분석하고 시각화합니다.

- 사고 및 보상 문의
- 상품 가입 및 해지
- 이체출금대출서비스
- 잔고 및 거래내역

In [ ]:
# 필요한 라이브러리 import
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from collections import Counter
import os
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 그래프 저장 폴더 생성
output_dir = '의도분석_그래프'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"폴더 생성: {output_dir}")
else:
    print(f"기존 폴더 사용: {output_dir}")

print("라이브러리 로드 완료")

## 1. 데이터 로드 함수

In [ ]:
def load_and_analyze_intent(json_file, category_name):
    """
    JSON 파일을 로드하고 고객의도와 상담사의도를 분석합니다.
    
    Parameters:
    - json_file: JSON 파일 경로
    - category_name: 카테고리 이름
    
    Returns:
    - customer_intent_top10: 고객의도 상위 10개
    - agent_intent_top10: 상담사의도 상위 10개
    """
    print(f"\n{'='*60}")
    print(f"처리 중: {category_name}")
    print(f"파일: {json_file}")
    print('='*60)
    
    # JSON 파일 로드
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except:
        with open(json_file, 'r', encoding='cp949') as f:
            data = json.load(f)
    
    # DataFrame 생성
    df = pd.DataFrame(data)
    print(f"전체 발화 수: {len(df):,}")
    
    # 고객의도 추출 (빈 값 제외)
    customer_intents = df[df['고객의도'] != '']['고객의도'].tolist()
    customer_intent_counts = Counter(customer_intents)
    customer_intent_top10 = customer_intent_counts.most_common(10)
    
    print(f"고객의도 종류: {len(customer_intent_counts)}개")
    print(f"고객의도 발화 수: {len(customer_intents):,}개")
    
    # 상담사의도 추출 (빈 값 제외)
    agent_intents = df[df['상담사의도'] != '']['상담사의도'].tolist()
    agent_intent_counts = Counter(agent_intents)
    agent_intent_top10 = agent_intent_counts.most_common(10)
    
    print(f"상담사의도 종류: {len(agent_intent_counts)}개")
    print(f"상담사의도 발화 수: {len(agent_intents):,}개")
    
    return customer_intent_top10, agent_intent_top10

print("함수 정의 완료")

## 2. 각 카테고리 데이터 로드 및 분석

In [ ]:
# 4개 카테고리 파일 정보
categories = [
    {
        'name': '사고 및 보상 문의',
        'file': '민원(콜센터) 질의응답_금융보험_사고 및 보상 문의_Training.json'
    },
    {
        'name': '상품 가입 및 해지',
        'file': '민원(콜센터) 질의응답_금융보험_상품 가입 및 해지_Training.json'
    },
    {
        'name': '이체출금대출서비스',
        'file': '민원(콜센터) 질의응답_금융보험_이체출금대출서비스_Training.json'
    },
    {
        'name': '잔고 및 거래내역',
        'file': '민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json'
    }
]

# 각 카테고리 데이터 로드 및 분석
results = {}

for category in categories:
    customer_top10, agent_top10 = load_and_analyze_intent(
        category['file'], 
        category['name']
    )
    results[category['name']] = {
        'customer': customer_top10,
        'agent': agent_top10
    }

print("\n전체 카테고리 데이터 로드 완료!")

## 3. 카테고리별 의도 분석 시각화

### 3-1. 사고 및 보상 문의

In [ ]:
# 사고 및 보상 문의
category_name = '사고 및 보상 문의'
customer_data = results[category_name]['customer']
agent_data = results[category_name]['agent']

# 데이터 준비
customer_labels = [item[0] for item in customer_data]
customer_values = [item[1] for item in customer_data]

agent_labels = [item[0] for item in agent_data]
agent_values = [item[1] for item in agent_data]

# 그래프 생성 (좌우 배치)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle(f'{category_name} - 의도 분포 (상위 10개)', fontsize=16, fontweight='bold')

# 왼쪽: 고객의도
colors1 = plt.cm.Blues(range(len(customer_labels), 0, -1))
ax1.barh(customer_labels[::-1], customer_values[::-1], color=colors1)
ax1.set_xlabel('발화 수', fontsize=12)
ax1.set_title('고객의도 (Top 10)', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(customer_values[::-1]):
    ax1.text(v + max(customer_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

# 오른쪽: 상담사의도
colors2 = plt.cm.Greens(range(len(agent_labels), 0, -1))
ax2.barh(agent_labels[::-1], agent_values[::-1], color=colors2)
ax2.set_xlabel('발화 수', fontsize=12)
ax2.set_title('상담사의도 (Top 10)', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(agent_values[::-1]):
    ax2.text(v + max(agent_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, f'{category_name}_의도분석.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

# 상세 데이터 출력
print(f"\n{category_name} - 고객의도 상위 10개:")
for i, (intent, count) in enumerate(customer_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

print(f"\n{category_name} - 상담사의도 상위 10개:")
for i, (intent, count) in enumerate(agent_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

### 3-2. 상품 가입 및 해지

In [ ]:
# 상품 가입 및 해지
category_name = '상품 가입 및 해지'
customer_data = results[category_name]['customer']
agent_data = results[category_name]['agent']

# 데이터 준비
customer_labels = [item[0] for item in customer_data]
customer_values = [item[1] for item in customer_data]

agent_labels = [item[0] for item in agent_data]
agent_values = [item[1] for item in agent_data]

# 그래프 생성 (좌우 배치)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle(f'{category_name} - 의도 분포 (상위 10개)', fontsize=16, fontweight='bold')

# 왼쪽: 고객의도
colors1 = plt.cm.Oranges(range(len(customer_labels), 0, -1))
ax1.barh(customer_labels[::-1], customer_values[::-1], color=colors1)
ax1.set_xlabel('발화 수', fontsize=12)
ax1.set_title('고객의도 (Top 10)', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(customer_values[::-1]):
    ax1.text(v + max(customer_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

# 오른쪽: 상담사의도
colors2 = plt.cm.Purples(range(len(agent_labels), 0, -1))
ax2.barh(agent_labels[::-1], agent_values[::-1], color=colors2)
ax2.set_xlabel('발화 수', fontsize=12)
ax2.set_title('상담사의도 (Top 10)', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(agent_values[::-1]):
    ax2.text(v + max(agent_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, f'{category_name}_의도분석.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

# 상세 데이터 출력
print(f"\n{category_name} - 고객의도 상위 10개:")
for i, (intent, count) in enumerate(customer_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

print(f"\n{category_name} - 상담사의도 상위 10개:")
for i, (intent, count) in enumerate(agent_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

### 3-3. 이체출금대출서비스

In [ ]:
# 이체출금대출서비스
category_name = '이체출금대출서비스'
customer_data = results[category_name]['customer']
agent_data = results[category_name]['agent']

# 데이터 준비
customer_labels = [item[0] for item in customer_data]
customer_values = [item[1] for item in customer_data]

agent_labels = [item[0] for item in agent_data]
agent_values = [item[1] for item in agent_data]

# 그래프 생성 (좌우 배치)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle(f'{category_name} - 의도 분포 (상위 10개)', fontsize=16, fontweight='bold')

# 왼쪽: 고객의도
colors1 = plt.cm.Reds(range(len(customer_labels), 0, -1))
ax1.barh(customer_labels[::-1], customer_values[::-1], color=colors1)
ax1.set_xlabel('발화 수', fontsize=12)
ax1.set_title('고객의도 (Top 10)', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(customer_values[::-1]):
    ax1.text(v + max(customer_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

# 오른쪽: 상담사의도
colors2 = plt.cm.YlOrBr(range(len(agent_labels), 0, -1))
ax2.barh(agent_labels[::-1], agent_values[::-1], color=colors2)
ax2.set_xlabel('발화 수', fontsize=12)
ax2.set_title('상담사의도 (Top 10)', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(agent_values[::-1]):
    ax2.text(v + max(agent_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, f'{category_name}_의도분석.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

# 상세 데이터 출력
print(f"\n{category_name} - 고객의도 상위 10개:")
for i, (intent, count) in enumerate(customer_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

print(f"\n{category_name} - 상담사의도 상위 10개:")
for i, (intent, count) in enumerate(agent_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

### 3-4. 잔고 및 거래내역

In [ ]:
# 잔고 및 거래내역
category_name = '잔고 및 거래내역'
customer_data = results[category_name]['customer']
agent_data = results[category_name]['agent']

# 데이터 준비
customer_labels = [item[0] for item in customer_data]
customer_values = [item[1] for item in customer_data]

agent_labels = [item[0] for item in agent_data]
agent_values = [item[1] for item in agent_data]

# 그래프 생성 (좌우 배치)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle(f'{category_name} - 의도 분포 (상위 10개)', fontsize=16, fontweight='bold')

# 왼쪽: 고객의도
colors1 = plt.cm.BuPu(range(len(customer_labels), 0, -1))
ax1.barh(customer_labels[::-1], customer_values[::-1], color=colors1)
ax1.set_xlabel('발화 수', fontsize=12)
ax1.set_title('고객의도 (Top 10)', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(customer_values[::-1]):
    ax1.text(v + max(customer_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

# 오른쪽: 상담사의도
colors2 = plt.cm.RdPu(range(len(agent_labels), 0, -1))
ax2.barh(agent_labels[::-1], agent_values[::-1], color=colors2)
ax2.set_xlabel('발화 수', fontsize=12)
ax2.set_title('상담사의도 (Top 10)', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(agent_values[::-1]):
    ax2.text(v + max(agent_values)*0.01, i, f'{v:,}', va='center', fontsize=10)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, f'{category_name}_의도분석.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

# 상세 데이터 출력
print(f"\n{category_name} - 고객의도 상위 10개:")
for i, (intent, count) in enumerate(customer_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

print(f"\n{category_name} - 상담사의도 상위 10개:")
for i, (intent, count) in enumerate(agent_data, 1):
    print(f"{i:2d}. {intent:20s} : {count:5,}개")

## 4. 전체 카테고리 통합 비교 (선택)

In [ ]:
# 4개 카테고리를 한 화면에 모두 표시 (2x2 그리드) - 고객의도
fig = plt.figure(figsize=(20, 16))
fig.suptitle('금융보험 4개 카테고리 - 고객의도 분포 비교 (상위 10개)', fontsize=18, fontweight='bold', y=0.995)

color_maps = ['Blues', 'Oranges', 'Reds', 'BuPu']

for idx, (category_name, color_map) in enumerate(zip(results.keys(), color_maps), 1):
    ax = plt.subplot(2, 2, idx)
    
    customer_data = results[category_name]['customer']
    labels = [item[0] for item in customer_data]
    values = [item[1] for item in customer_data]
    
    colors = plt.cm.get_cmap(color_map)(range(len(labels), 0, -1))
    ax.barh(labels[::-1], values[::-1], color=colors)
    ax.set_xlabel('발화 수', fontsize=11)
    ax.set_title(f'{category_name}\n고객의도 (Top 10)', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    for i, v in enumerate(values[::-1]):
        ax.text(v + max(values)*0.01, i, f'{v:,}', va='center', fontsize=9)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, '전체_고객의도_통합비교.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

# 상담사의도 통합 비교
fig = plt.figure(figsize=(20, 16))
fig.suptitle('금융보험 4개 카테고리 - 상담사의도 분포 비교 (상위 10개)', fontsize=18, fontweight='bold', y=0.995)

color_maps = ['Greens', 'Purples', 'YlOrBr', 'RdPu']

for idx, (category_name, color_map) in enumerate(zip(results.keys(), color_maps), 1):
    ax = plt.subplot(2, 2, idx)
    
    agent_data = results[category_name]['agent']
    labels = [item[0] for item in agent_data]
    values = [item[1] for item in agent_data]
    
    colors = plt.cm.get_cmap(color_map)(range(len(labels), 0, -1))
    ax.barh(labels[::-1], values[::-1], color=colors)
    ax.set_xlabel('발화 수', fontsize=11)
    ax.set_title(f'{category_name}\n상담사의도 (Top 10)', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    for i, v in enumerate(values[::-1]):
        ax.text(v + max(values)*0.01, i, f'{v:,}', va='center', fontsize=9)

plt.tight_layout()

# JPG 파일로 저장
output_file = os.path.join(output_dir, '전체_상담사의도_통합비교.jpg')
plt.savefig(output_file, format='jpg', dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {output_file}")

plt.show()

## 5. 분석 요약

In [ ]:
# 전체 요약 출력
print("="*80)
print("금융보험 카테고리별 의도 분석 요약")
print("="*80)

for category_name in results.keys():
    print(f"\n[{category_name}]")
    print("-" * 60)
    
    customer_top1 = results[category_name]['customer'][0]
    agent_top1 = results[category_name]['agent'][0]
    
    print(f"가장 많은 고객의도: {customer_top1[0]} ({customer_top1[1]:,}개)")
    print(f"가장 많은 상담사의도: {agent_top1[0]} ({agent_top1[1]:,}개)")
    
    total_customer = sum([item[1] for item in results[category_name]['customer']])
    total_agent = sum([item[1] for item in results[category_name]['agent']])
    
    print(f"고객의도 상위 10개 합계: {total_customer:,}개")
    print(f"상담사의도 상위 10개 합계: {total_agent:,}개")

print("\n" + "="*80)
print("분석 완료!")
print("="*80)

# 저장된 그래프 파일 목록
print("\n저장된 그래프 파일:")
print("-" * 60)
saved_files = [
    '사고 및 보상 문의_의도분석.jpg',
    '상품 가입 및 해지_의도분석.jpg',
    '이체출금대출서비스_의도분석.jpg',
    '잔고 및 거래내역_의도분석.jpg',
    '전체_고객의도_통합비교.jpg',
    '전체_상담사의도_통합비교.jpg'
]

for i, filename in enumerate(saved_files, 1):
    filepath = os.path.join(output_dir, filename)
    if os.path.exists(filepath):
        file_size = os.path.getsize(filepath) / 1024  # KB
        print(f"{i}. {filename} ({file_size:.1f} KB)")
    else:
        print(f"{i}. {filename} (파일 없음)")

print(f"\n모든 그래프가 '{output_dir}' 폴더에 저장되었습니다!")